In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%load_ext rpy2.ipython

In [2]:
from kaggle.house_prices import helpers
from kaggle.house_prices import outliers

combined_dataset = helpers.load_data()
training_dataset = combined_dataset.query("dataSource == 'train'")

In [114]:
import pandas as pd

df = combined_dataset

dict((c, 1)for c in list(df))

{'1stFlrSF': 1,
 '2ndFlrSF': 1,
 '3SsnPorch': 1,
 'Alley': 1,
 'BedroomAbvGr': 1,
 'BldgType': 1,
 'BsmtCond': 1,
 'BsmtExposure': 1,
 'BsmtFinSF1': 1,
 'BsmtFinSF2': 1,
 'BsmtFinType1': 1,
 'BsmtFinType2': 1,
 'BsmtFullBath': 1,
 'BsmtHalfBath': 1,
 'BsmtQual': 1,
 'BsmtUnfSF': 1,
 'CentralAir': 1,
 'Condition1': 1,
 'Condition2': 1,
 'Electrical': 1,
 'EnclosedPorch': 1,
 'ExterCond': 1,
 'ExterQual': 1,
 'Exterior1st': 1,
 'Exterior2nd': 1,
 'Fence': 1,
 'FireplaceQu': 1,
 'Fireplaces': 1,
 'Foundation': 1,
 'FullBath': 1,
 'Functional': 1,
 'GarageArea': 1,
 'GarageCars': 1,
 'GarageCond': 1,
 'GarageFinish': 1,
 'GarageQual': 1,
 'GarageType': 1,
 'GarageYrBlt': 1,
 'GrLivArea': 1,
 'HalfBath': 1,
 'Heating': 1,
 'HeatingQC': 1,
 'HouseStyle': 1,
 'Id': 1,
 'KitchenAbvGr': 1,
 'KitchenQual': 1,
 'LandContour': 1,
 'LandSlope': 1,
 'LotArea': 1,
 'LotConfig': 1,
 'LotFrontage': 1,
 'LotShape': 1,
 'LowQualFinSF': 1,
 'MSSubClass': 1,
 'MSZoning': 1,
 'MasVnrArea': 1,
 'MasVnrType':